In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
import numpy as np
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer ## HAndle Missing Values
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
import xgboost as xb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from imblearn.combine import SMOTETomek
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedGroupKFold
import sweetviz as sv 
pd.set_option('display.max_columns',None)
plt.rcParams['figure.figsize']=(15,10)
from sklearn.feature_selection import mutual_info_classif  ## good for only classification problem 
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import OrdinalEncoder
from imblearn.over_sampling import RandomOverSampler

# Feature Engineering

In [2]:
df=pd.read_csv('og1.csv')

In [3]:
df.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Duration_hours,Duration_minutes,Journey_year,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute
0,IndiGo,Banglore,New Delhi,"['BLR ', ' DEL']",non-stop,No info,3897,2.0,50,2019,3,24,22,20,1,10
1,Air India,Kolkata,Banglore,"['CCU ', ' IXR ', ' BBI ', ' BLR']",2 stops,No info,7662,7.0,25,2019,5,1,5,50,13,15
2,Jet Airways,Delhi,Cochin,"['DEL ', ' LKO ', ' BOM ', ' COK']",2 stops,No info,13882,19.0,0,2019,6,9,9,25,4,25
3,IndiGo,Kolkata,Banglore,"['CCU ', ' NAG ', ' BLR']",1 stop,No info,6218,5.0,25,2019,5,12,18,5,23,30
4,IndiGo,Banglore,New Delhi,"['BLR ', ' NAG ', ' DEL']",1 stop,No info,13302,4.0,45,2019,3,1,16,50,21,35


In [4]:
df.drop(['Journey_year'],axis=1,inplace=True)

In [5]:
df.isnull().sum().sum()

2

In [6]:
df.dropna(inplace=True)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10459 entries, 0 to 10682
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Airline           10459 non-null  object 
 1   Source            10459 non-null  object 
 2   Destination       10459 non-null  object 
 3   Route             10459 non-null  object 
 4   Total_Stops       10459 non-null  object 
 5   Additional_Info   10459 non-null  object 
 6   Price             10459 non-null  int64  
 7   Duration_hours    10459 non-null  float64
 8   Duration_minutes  10459 non-null  int64  
 9   Journey_month     10459 non-null  int64  
 10  Journey_day       10459 non-null  int64  
 11  Dep_hour          10459 non-null  int64  
 12  Dep_minute        10459 non-null  int64  
 13  Arv_hour          10459 non-null  int64  
 14  Arv_minute        10459 non-null  int64  
dtypes: float64(1), int64(8), object(6)
memory usage: 1.3+ MB


In [9]:
df['Duration_hours']=df['Duration_hours'].astype('int16')

In [10]:
df.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Duration_hours,Duration_minutes,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute
0,IndiGo,Banglore,New Delhi,"['BLR ', ' DEL']",non-stop,No info,3897,2,50,3,24,22,20,1,10
1,Air India,Kolkata,Banglore,"['CCU ', ' IXR ', ' BBI ', ' BLR']",2 stops,No info,7662,7,25,5,1,5,50,13,15
2,Jet Airways,Delhi,Cochin,"['DEL ', ' LKO ', ' BOM ', ' COK']",2 stops,No info,13882,19,0,6,9,9,25,4,25
3,IndiGo,Kolkata,Banglore,"['CCU ', ' NAG ', ' BLR']",1 stop,No info,6218,5,25,5,12,18,5,23,30
4,IndiGo,Banglore,New Delhi,"['BLR ', ' NAG ', ' DEL']",1 stop,No info,13302,4,45,3,1,16,50,21,35


In [11]:
cat_feat=[feature for feature in df.columns if df[feature].dtype=='O']

In [12]:
for feature in cat_feat:
    print(df[feature].value_counts())
    print('************************')

Airline
Jet Airways                          3700
IndiGo                               2043
Air India                            1693
Multiple carriers                    1196
SpiceJet                              815
Vistara                               477
Air Asia                              318
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64
************************
Source
Delhi       4345
Kolkata     2860
Banglore    2177
Mumbai       696
Chennai      381
Name: count, dtype: int64
************************
Destination
Cochin       4345
Banglore     2860
Delhi        1265
New Delhi     912
Hyderabad     696
Kolkata       381
Name: count, dtype: int64
************************
Route
['DEL ', ' BOM ', ' COK']             2376
['BLR ', ' DEL']                      1534
['CCU ', ' BOM ', ' BLR']      

In [13]:
df.drop('Additional_Info',inplace=True,axis=1)

In [14]:
# df['Additional_Info']=df['Additional_Info'].str.replace('No Info','No info')

In [15]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,Duration_hours,Duration_minutes,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute
0,IndiGo,Banglore,New Delhi,"['BLR ', ' DEL']",non-stop,3897,2,50,3,24,22,20,1,10


In [16]:
df.reset_index(drop=True,inplace=True)

In [17]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,Duration_hours,Duration_minutes,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute
0,IndiGo,Banglore,New Delhi,"['BLR ', ' DEL']",non-stop,3897,2,50,3,24,22,20,1,10


In [18]:
df['Duration_hours']=df['Duration_hours']*60

In [19]:
df['Total_Duration']=df['Duration_hours']+df['Duration_minutes']

In [20]:
# df['Total_Duration']=df['Total_Duration']/60


In [21]:
# df['Total_Duration']=df['Total_Duration'].astype('float16')

In [22]:
df['Total_Duration']

0         170
1         445
2        1140
3         325
4         285
         ... 
10454     150
10455     155
10456     180
10457     160
10458     500
Name: Total_Duration, Length: 10459, dtype: int64

In [23]:
df.drop(['Duration_hours','Duration_minutes'],axis=1,inplace=True)

In [24]:
def list_cov(s):
    return eval(s)

In [25]:
df['Route']=df['Route'].apply(list_cov)

In [26]:
def len_cov(s):
    return len(s)

In [27]:
df['Route_count']=df['Route'].apply(len_cov)

In [28]:
df.drop('Route',axis=1,inplace=True)

In [29]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [30]:
df['Total_Stops'].replace({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4},inplace=True)

In [31]:
df.to_csv("gh.csv",index=False)

In [ ]:
# encoding 1 

In [334]:
one_hot_encoding=pd.get_dummies(df[['Airline','Source','Destination']])

df_encoded=pd.concat([one_hot_encoding,df],axis=1)

df_encoded.drop(['Airline','Source','Destination'],axis=1,inplace=True)


In [335]:
df_encoded

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Total_Stops,Additional_Info,Price,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute,Total_Duration,Route_Number
0,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,0,No info,3897,3,24,22,20,1,10,170,2
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,2,No info,7662,5,1,5,50,13,15,445,4
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,2,No info,13882,6,9,9,25,4,25,1140,4
3,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,1,No info,6218,5,12,18,5,23,30,325,3
4,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,1,No info,13302,3,1,16,50,21,35,285,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10454,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,0,No info,4107,4,9,19,55,22,25,150,2
10455,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,0,No info,4145,4,27,20,45,23,20,155,2
10456,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,0,No info,7229,4,27,8,20,11,20,180,2
10457,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,0,No info,12648,3,1,11,30,14,10,160,2


In [336]:
df_encoded.to_csv('f2.csv',index=False)

In [ ]:
## encoding 2

In [77]:
for feature in cat_feat:
    labels_ordered=df.groupby([feature])['Price'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df[feature]=df[feature].map(labels_ordered)

In [78]:
df.head()

,Airline,Source,Destination,Total_Stops,Price,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute,Total_Duration,Route_Number
0,3,2,5,0,3897,3,24,22,20,1,10,170,2
1,7,3,3,2,7662,5,1,5,50,13,15,445,4
2,10,4,4,2,13882,6,9,9,25,4,25,1140,4
3,3,3,3,1,6218,5,12,18,5,23,30,325,3
4,3,2,5,1,13302,3,1,16,50,21,35,285,3


In [79]:
df.to_csv('main.csv',index=False)

In [176]:
## encoding 3

In [177]:
le=LabelEncoder()

In [178]:
df["Airline"] = le.fit_transform(df["Airline"])

In [179]:
df['Source'] = le.fit_transform(df['Source'])

In [180]:
df['Destination'] = le.fit_transform(df['Destination'])

In [181]:
df['Total_Stops'].replace({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4},inplace=True)

In [182]:
df.head()

,Airline,Source,Destination,Total_Stops,Price,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute,Total_Duration,Route_Number
0,3,0,5,0,3897,3,24,22,20,1,10,170,2
1,1,3,0,2,7662,5,1,5,50,13,15,445,4
2,4,2,1,2,13882,6,9,9,25,4,25,1140,4
3,3,3,0,1,6218,5,12,18,5,23,30,325,3
4,3,0,5,1,13302,3,1,16,50,21,35,285,3


In [229]:
df.to_csv('bo.csv',index=False)

In [12]:
import xgboost as xgb
from xgboost import XGBRegressor

# Model_training

In [80]:
df=pd.read_csv('main.csv')

In [81]:
df.head()

,Airline,Source,Destination,Total_Stops,Price,Journey_month,Journey_day,Dep_hour,Dep_minute,Arv_hour,Arv_minute,Total_Duration,Route_Number
0,3,2,5,0,3897,3,24,22,20,1,10,170,2
1,7,3,3,2,7662,5,1,5,50,13,15,445,4
2,10,4,4,2,13882,6,9,9,25,4,25,1140,4
3,3,3,3,1,6218,5,12,18,5,23,30,325,3
4,3,2,5,1,13302,3,1,16,50,21,35,285,3


In [82]:
X=df.drop('Price',axis=1)
y=df.Price

In [106]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.3)

In [107]:
scaler=MaxAbsScaler()

In [108]:
scaler.fit(X_train)

MaxAbsScaler()

In [109]:
from sklearn.preprocessing import MaxAbsScaler

In [110]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [111]:
models=[DecisionTreeRegressor(),RandomForestRegressor(),LinearRegression(),SVR(),GradientBoostingRegressor(),XGBRegressor()]

In [112]:
def model_train(X_train,X_test,y_train,y_test,models):
    for model in models:
        
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        print(model)
        print(r2_score(y_test,y_pred))
        print(mean_squared_error(y_test,y_pred))
        print('********************************')

In [113]:
model_train(X_train_scaled,X_test_scaled,y_train,y_test,models)

DecisionTreeRegressor()
0.7235538927969821
5809267.461521492
********************************
RandomForestRegressor()
0.8311557605847982
3548110.5377973225
********************************
LinearRegression()
0.569337971115738
9049976.99776928
********************************
SVR()
0.08473759762371169
19233420.020538885
********************************
GradientBoostingRegressor()
0.7841350690262439
4536208.276821542
********************************
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
    

In [98]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [99]:
xgbreg = XGBRegressor()

In [100]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3, 4],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1],
}

In [101]:
grid_CV = RandomizedSearchCV(estimator=xgbreg,param_distributions=param_grid,cv=5,n_jobs=-1,verbose=1)

In [102]:
grid_CV.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8,
                                                             0.9],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 200, 300],
                                        'reg_alpha': [0, 0.1, 0.5, 1],
                                        'reg_lambda': [0, 0.1, 0.5, 1],
                                        'subsample': [0.6, 0.7, 0.8, 0.9]},
                   verbose=1)

In [103]:
grid_CV.best_params_

{'subsample': 0.9,
 'reg_lambda': 0,
 'reg_alpha': 1,
 'n_estimators': 300,
 'min_child_weight': 3,
 'max_depth': 6,
 'learning_rate': 0.2,
 'gamma': 0.2,
 'colsample_bytree': 0.6}

In [104]:
y_pred = grid_CV.predict(X_test)

In [105]:
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R^2 Score: {r2_score(y_test,y_pred):.4f}")

MAE: 1197.1683
MSE: 3415842.8436
RMSE: 1848.1999
R^2 Score: 0.8410


In [294]:
from lightgbm import LGBMRegressor
gbm_regressor = LGBMRegressor()
gbm_regressor.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000258 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 379
[LightGBM] [Info] Number of data points in the train set: 7321, number of used features: 28
[LightGBM] [Info] Start training from score 9076.882393


LGBMRegressor()

In [295]:

y_test_pred = gbm_regressor.predict(X_test)

In [296]:
print("R-squared: ", r2_score(y_test, y_test_pred))

R-squared:  0.8249121181744503


In [16]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500,50],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [None, 10, 20, 30, 40, 50,25,27],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10, 15,20,25],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4, 6,0],  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]  # Method of selecting samples for training each tree
}

In [17]:
grid_CV = RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=param_grid,cv=5,n_jobs=-1,verbose=1)

In [18]:
grid_CV.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30, 40, 50,
                                                      25, 27],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 0],
                                        'min_samples_split': [2, 5, 10, 15, 20,
                                                              25],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 50]},
                   verbose=1)

In [19]:
grid_CV.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 27,
 'bootstrap': False}

In [20]:
y_pred = grid_CV.predict(X_test)

In [21]:
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R^2 Score: {r2_score(y_test,y_pred):.4f}")

MAE: 742.0488
MSE: 2033608.8348
RMSE: 1426.0466
R^2 Score: 0.9053
